# Azure AI Search su NVIDIA NIM ir LlamaIndex integracija

Šiame užrašų knygelėje parodysime, kaip pasinaudoti NVIDIA dirbtinio intelekto modeliais ir LlamaIndex, kad sukurtume galingą Retrieval-Augmented Generation (RAG) grandinę. Naudosime NVIDIA LLM ir įterpinius, integruosime juos su Azure AI Search kaip vektorinę saugyklą ir atliksime RAG, kad pagerintume paieškos kokybę ir efektyvumą.

## Privalumai
- **Skalabilumas**: Pasitelkite NVIDIA didelius kalbos modelius ir Azure AI Search, kad užtikrintumėte mastelį ir efektyvų informacijos gavimą.
- **Kaštų efektyvumas**: Optimizuokite paiešką ir informacijos gavimą naudodami efektyvų vektorinį saugojimą ir hibridines paieškos technikas.
- **Aukštas našumas**: Sujunkite galingus LLM su vektorizuota paieška, kad gautumėte greitesnius ir tikslesnius atsakymus.
- **Kokybė**: Išlaikykite aukštą paieškos kokybę, pagrįsdami LLM atsakymus su susijusiais surastais dokumentais.

## Reikalavimai
- 🐍 Python 3.9 arba naujesnė
- 🔗 [Azure AI Search paslauga](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API raktas prieigai prie NVIDIA LLM ir įterpinių per NVIDIA NIM mikroservisus

## Aprėptos funkcijos
- ✅ NVIDIA LLM integracija (naudosime [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ NVIDIA įterpiniai (naudosime [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Azure AI Search pažangūs atgavimų režimai
- ✅ Dokumentų indeksavimas su LlamaIndex
- ✅ RAG naudojant Azure AI Search ir LlamaIndex su NVIDIA LLMs

Pradėkime!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Įdiegimas ir reikalavimai
Sukurkite Python aplinką naudodami Python versiją >3.10.

## Pradėkite!


Norėdami pradėti, kad galėtumėte naudoti NVIDIA AI Foundation modelius, jums reikalingas `NVIDIA_API_KEY`:
1) Sukurkite nemokamą paskyrą su [NVIDIA](https://build.nvidia.com/explore/discover).
2) Spustelėkite pasirinktą modelį.
3) Skiltyje Input pasirinkite skirtuką Python, spustelėkite **Gauti API raktą** ir tada spustelėkite **Generuoti raktą**.
4) Nukopijuokite ir išsaugokite sugeneruotą raktą kaip NVIDIA_API_KEY. Nuo to momento turėtumėte turėti prieigą prie galinių taškų.


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## RAG pavyzdys naudojant LLM ir įterpimus
### 1) Inicializuokite LLM
`llama-index-llms-nvidia`, dar žinomas kaip NVIDIA LLM jungtis, leidžia prisijungti prie suderinamų modelių, esančių NVIDIA API kataloge, ir generuoti iš jų. Žr. čia pokalbių užbaigimo modelių sąrašą: https://build.nvidia.com/search?term=Text-to-Text

Čia naudosime **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inicializuokite įterpimą
`llama-index-embeddings-nvidia`, taip pat žinomas kaip NVIDIA Embeddings jungtis, leidžia prisijungti prie ir generuoti iš suderinamų modelių, esančių NVIDIA API kataloge. Mes pasirinkome `nvidia/nv-embedqa-e5-v5` kaip įterpimo modelį. Peržiūrėkite čia tekstinių įterpimo modelių sąrašą: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

### 3) Sukurkite Azure AI Search vektorinę saugyklą


In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

### 4) Įkelti, suskaidyti ir nusiųsti dokumentus


In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Sukurkite užklausų variklį, kad galėtumėte užduoti klausimus apie savo duomenis

Čia pateikta užklausa, naudojant tik vektorinę paiešką Azure AI Search ir pagrindžiant atsakymą mūsų LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Štai užklausa, naudojanti hibridinę paiešką Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Vektorinės paieškos analizė
LLM atsakymas tiksliai atspindi pagrindines ekonomines pasekmes, paminėtas šaltinio tekste, susijusias su Rusijos akcijų rinka. Konkrečiai, jame teigiama, kad Rusijos akcijų rinka patyrė reikšmingą nuosmukį, praradusi 40% savo vertės, ir kad prekyba buvo sustabdyta dėl susiklosčiusios situacijos. Šis atsakymas gerai atitinka šaltinyje pateiktą informaciją, nurodydamas, kad LLM teisingai identifikavo ir apibendrino susijusias detales apie akcijų rinkos poveikį, kilusį dėl Rusijos veiksmų ir įvestų sankcijų.

#### Šaltinio mazgų komentaras
Šaltinio mazgai pateikia išsamų aprašymą apie ekonomines pasekmes, kurias Rusija patyrė dėl tarptautinių sankcijų. Tekste pabrėžiama, kad Rusijos akcijų rinka prarado 40% savo vertės, ir prekyba buvo sustabdyta. Be to, minima ir kitų ekonominių pasekmių, tokių kaip rublio nuvertėjimas ir platesnė Rusijos ekonomikos izoliacija. LLM atsakymas efektyviai išskyrė svarbiausius šių mazgų punktus, sutelkiant dėmesį į akcijų rinkos poveikį, kaip nurodyta užklausoje.


Dabar pažvelkime į užklausą, kurioje Hybrid Search nepateikia gerai pagrįsto atsakymo:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Hibridinė paieška: LLM atsako analizė
LLM atsakymas hibridinės paieškos pavyzdyje nurodo, kad pateiktame kontekste nėra nurodyta tiksli Rusijos invazijos į Ukrainą data. Šis atsakymas rodo, kad LLM naudoja šaltinio dokumentuose esančią informaciją, tačiau pripažįsta, jog tekste trūksta tikslių detalių.

Atsakymas teisingai nurodo, kad kontekste minimi su Rusijos agresija susiję įvykiai, tačiau jis nenustato tikslios invazijos datos. Tai atspindi LLM gebėjimą suprasti pateiktą informaciją ir tuo pačiu atpažinti spragas turinyje. LLM veiksmingai skatina vartotoją ieškoti išorinių šaltinių arba istorinių įrašų dėl tikslios datos, rodydamas atsargumą, kai informacija yra neišsami.

### Šaltinių mazgų analizė
Hibridinės paieškos pavyzdyje šaltinių mazguose yra ištraukos iš kalbos, kurioje aptariamas JAV atsakas į Rusijos veiksmus Ukrainoje. Šie mazgai pabrėžia platesnį geopolitinį poveikį ir žingsnius, kurių ėmėsi JAV ir jos sąjungininkai reaguodami į invaziją, tačiau jie nepaminėja konkrečios invazijos datos. Tai atitinka LLM atsakymą, kuris teisingai nurodo, jog kontekste trūksta tikslios datos informacijos.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Hibridinis su perrikiavimu: LLM atsakymo analizė
Šiame Hibridinis su perrikiavimu pavyzdyje LLM atsakymas suteikia papildomą kontekstą pažymėdamas, kad įvykis įvyko šešias dienas prieš kalbos pasakymą. Tai rodo, kad LLM sugeba išvesti invazijos datą remdamasis kalbos laiku, nors jam vis tiek reikia žinoti tikslią kalbos datą, kad užtikrintų tikslumą.

Šis atsakymas demonstruoja patobulėjusį gebėjimą naudoti kontekstines užuominas, kad pateiktų informatyvesnį atsakymą. Tai pabrėžia perrikiavimo pranašumą, kai LLM gali pasiekti ir prioritetizuoti labiau aktualią informaciją, kad pateiktų artimesnę pageidaujamos detalės apytikrę (t. y. invazijos datą).

### Šaltinių mazgų analizė
Šio pavyzdžio šaltinių mazgai įtraukia nuorodas į Rusijos invazijos laiką, konkrečiai nurodydami, kad ji įvyko šešias dienas prieš kalbą. Nors tiksli data vis dar nėra aiškiai pateikta, mazgai suteikia laikinį kontekstą, kuris leidžia LLM pateikti niuansuotesnį atsakymą. Šios detalės įtraukimas iliustruoja, kaip perrikiavimas gali pagerinti LLM gebėjimą ištraukti ir išvesti informaciją iš pateikto konteksto, kas lemia tikslesnį ir informatyvesnį atsakymą.


**Pastaba:**
Šiame užrašų knygelėje naudojome NVIDIA NIM mikroservisus iš NVIDIA API katalogo.
Aukščiau paminėtos API, `NVIDIA (llms)`, `NVIDIAEmbedding`, ir [Azure AI Search semantinis hibridinis atgavimas (su įmontuotu perreitingavimu)](https://learn.microsoft.com/azure/search/semantic-search-overview). Atkreipkite dėmesį, kad aukščiau minėtos API taip pat gali palaikyti savarankiškai talpinamus mikroservisus. 

**Pavyzdys:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
Atsakomybės apribojimas:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, atkreipkite dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turi būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama kreiptis į profesionalius žmogaus vertėjus. Mes neatsakome už jokius nesusipratimus ar neteisingus aiškinimus, kilusius dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
